In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

# Load the full corpus
dataset = load_dataset("mohres/The_Arabic_E-Book_Corpus", split="train")

# Print metadata for the first few books to choose one
for i in range(500):
    print(dataset[i]['title'], dataset[i]['author'], dataset[i]["booknr"])

README.md:   0%|          | 0.00/3.56k [00:00<?, ?B/s]

data-00000-of-00002.arrow:   0%|          | 0.00/347M [00:00<?, ?B/s]

data-00001-of-00002.arrow:   0%|          | 0.00/346M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1745 [00:00<?, ? examples/s]

لؤلؤة الحب هربرت جورج ويلز 13028368
مغامرة العميل المرموق آرثر كونان دويل 13035305
ثلاثة دروس في ديكارت ألكسندر كواريه 13050381
كشف المُخبَّا عن فنون أوربا أحمد فارس الشدياق 13068491
حقائق الإسلام وأباطيل خصومه عباس محمود العقاد 13082050
مدينة زحلة عيسى إسكندر المعلوف 13149739
لاسلكيًّا روديارد كبلينج 13153615
مغامرة راكبة الدراجة في الطريق المهجور آرثر كونان دويل 13181417
الفاروق عمر محمد حسين هيكل 13538483
إبراهيم الثاني إبراهيم عبد القادر المازني 13572470
الطبيعة وما بعد الطبيعة: المادة . الحياة . الله يوسف كرم 13572791
الحجاج بن يوسف جُرجي زيدان 13595164
المسألة السودانية عمر طوسون 13603829
غَادَةُ كرْبَلاء جُرجي زيدان 13604281
أبيض الناب جاك لندن 13639135
أنت من فعلها! إدجار آلان بو 13649293
البدوية إبراهيم رمزي 13680727
قصة الأيام القادمة هربرت جورج ويلز 13736816
ألف ليلة وليلة ألف ليلة وليلة 13815802
نماذج بشرية أحمد رضا حوحو 13919315
تذكار الصبا: ذكرى ١٩ مارس محمد لطفي جمعة 13919490
الإسلام والحضارة الإنسانية عباس محمود العقاد 13920718
رواية الشقيقتين هنري لامنس 13925094
فَلسَف

In [ ]:
x =dataset["booknr"]
i=0
for nr in dataset["booknr"]:
  if nr == 13815802:
    book = dataset[i]
    text = book['text']
    # Save the text to a file
    with open("selected_book.txt", "w", encoding="utf-8") as f:
        f.write(text)
  i +=1

In [ ]:
import re

def split_into_paragraphs(input_file, output_file):
    # Read the text from the input file with UTF-8 encoding to support Arabic characters
    with open(input_file, 'r', encoding='utf-8') as f:
        text = f.read()

    # Split the text into sentences using a regex pattern
    # The pattern (?<=[.؟!])\s+ splits on whitespace that follows a period (.), Arabic question mark (؟), or exclamation point (!)
    # This keeps the punctuation with each sentence
    sentences = [s for s in re.split(r'(?<=[.؟!])\s+', text) if s.strip()]

    # Group sentences into paragraphs
    paragraphs = []
    for i in range(0, len(sentences), 4):
        # Take a slice of up to 4 sentences
        paragraph = sentences[i:i+3]
        # If this is not the first group and it has only 1 sentence, append it to the previous paragraph
        if i > 0 and len(paragraph) == 1:
            paragraphs[-1].append(paragraph[0])
        else:
            # Otherwise, add the new paragraph to the list
            paragraphs.append(paragraph)

    # Convert each paragraph (list of sentences) into a single string by joining sentences with spaces
    paragraph_texts = [' '.join(para) for para in paragraphs]

    # Write the paragraphs to the output file, each separated by a blank line
    with open(output_file, 'w', encoding='utf-8') as f:
        for para_text in paragraph_texts:
            f.write(para_text + '\n\n')

if __name__ == '__main__':
    # Define input and output filenames
    input_file = 'selected_book.txt'
    output_file = 'splitted_book.txt'
    # Call the function to process the file
    split_into_paragraphs(input_file, output_file)

In [11]:
from sentence_transformers import SentenceTransformer
import numpy as np
import os
from tqdm import tqdm

def generate_embeddings(input_file, output_file, batch_size=32, model_name='sentence-transformers/LaBSE'):
    """
    Generate sentence embeddings for paragraphs in an input file using a multilingual model optimized for Arabic.

    Args:
        input_file (str): Path to the input file containing paragraphs.
        output_file (str): Path to save the embeddings as a NumPy array.
        batch_size (int): Number of paragraphs to process per batch (default: 32).
        model_name (str): Name of the SentenceTransformer model (default: 'sentence-transformers/LaBSE').

    Returns:
        None

    Raises:
        FileNotFoundError: If the input file does not exist.
        ValueError: If no valid paragraphs are found or embeddings are invalid.
    """
    if not os.path.exists(input_file):
        raise FileNotFoundError(f"Input file {input_file} not found.")

    print(f"Loading model: {model_name}")
    model = SentenceTransformer(model_name)
    print("Model loaded.")

    with open(input_file, 'r', encoding='utf-8') as f:
        content = f.read()

    paragraphs = [para.strip() for para in content.split('\n\n') if para.strip()]
    if not paragraphs:
        raise ValueError("No valid paragraphs found in the input file.")
    print(f"File read, number of paragraphs: {len(paragraphs)}")

    embeddings = []
    for i in tqdm(range(0, len(paragraphs), batch_size), desc="Generating embeddings", total=(len(paragraphs) // batch_size + 1)):
        batch = paragraphs[i:i + batch_size]
        batch_embeddings = model.encode(batch, normalize_embeddings=True)
        embeddings.extend(batch_embeddings)

    embeddings = np.array(embeddings)
    if embeddings.shape[0] != len(paragraphs):
        raise ValueError(f"Expected {len(paragraphs)} embeddings, got {embeddings.shape[0]}.")
    if np.any(np.isnan(embeddings)):
        raise ValueError("Embeddings contain NaN values.")

    np.save(output_file, embeddings)
    print(f"Embeddings saved to {output_file}, shape: {embeddings.shape}")




input_file = '/content/splitted_book.txt'
output_file = 'embeddings.npy'
# Generate embeddings
generate_embeddings(input_file, output_file)

Loading model: sentence-transformers/LaBSE


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

2_Dense/model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

2_Dense/pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

Model loaded.
File read, number of paragraphs: 467


Generating embeddings: 100%|██████████| 15/15 [00:02<00:00,  5.12it/s]

Embeddings saved to embeddings.npy, shape: (467, 768)


In [12]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 26.6 MB/s eta 0:00:00


In [13]:
!pip list | grep faiss

faiss-cpu                             1.11.0


In [14]:
import faiss


import numpy as np

# Load the embeddings
embeddings = np.load('embeddings.npy').astype('float32')


# Get the dimensionality of the vectors (e.g., 512)
dimension = embeddings.shape[1]

# Create the FAISS index
index = faiss.IndexFlatL2(dimension)

# Add embeddings to the index
index.add(embeddings)

# Save the index to a file
faiss.write_index(index, 'faiss_index.bin')

In [18]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os

# Load paragraphs
with open('splitted_book.txt', 'r', encoding='utf-8') as f:
    content = f.read()
paragraphs = [para.strip() for para in content.split('\n\n') if para.strip()]

# Load embeddings and FAISS index
embeddings = np.load('embeddings.npy').astype('float32')
index = faiss.read_index('faiss_index.bin')

# Load the SentenceTransformers model
model = SentenceTransformer('sentence-transformers/LaBSE')

# Set up TF-IDF vectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(paragraphs)


In [19]:
def classical_search(query, top_k=5):
    query_vec = vectorizer.transform([query])
    similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()
    top_indices = similarities.argsort()[-top_k:][::-1]
    return [(paragraphs[i], similarities[i]) for i in top_indices]

def semantic_search(query, top_k=5):
    query_embedding = model.encode([query], convert_to_tensor=False).astype('float32')
    distances, indices = index.search(query_embedding, top_k)
    return [(paragraphs[i], distances[0][j]) for j, i in enumerate(indices[0])]

def display_results(classical_results, semantic_results):
    print("\nنتائج البحث")

    for i in range(5):
        # Classical search result
        c_text = classical_results[i][0]
        c_score = classical_results[i][1]
        # Semantic search result
        s_text = semantic_results[i][0]
        s_distance = semantic_results[i][1]

        print("-"*20)
        print(f"\nالنتائج {i + 1}:")
        print(f"\nClassical search result: {c_text}")
        print(f"Score: {c_score}")
        print(f"\nSemantic search result: {s_text}")
        print(f"Distance: {s_distance}")
        print("-"*20)


In [22]:
while True:
    query = input("\nأدخل استعلامك بالعربية (أو 'quit' للخروج): ")
    if query.lower() == 'quit':
        break
    if not query.strip():
        print("الرجاء إدخال استعلام صالح.")
        continue

    # Perform searches
    classical_results = classical_search(query)
    semantic_results = semantic_search(query)

    # Display results
    print(f"\nنتائج البحث للاستعلام: {query}")
    display_results(classical_results, semantic_results)


أدخل استعلامك بالعربية (أو 'quit' للخروج): من هم  شهريار وشاهزمان؟

نتائج البحث للاستعلام: من هم  شهريار وشاهزمان؟

نتائج البحث
--------------------

النتائج 1:

Classical search result: يُحكى أنه في قديم الزمان عاش ملكان عظيمان شقيقان يُدعيان شهريار وشاهزمان. كان شهريار هو الأكبر سنًّا، وامتد سلطانه إلى أقاصي الأرض. شعر شهريار برغبة في رؤية أخيه، فطلب من كبير وزرائه الذهاب لإحضاره إليه.
Score: 0.32989686491101883

Semantic search result: قصة الملك شهريار وشهرزاد
Distance: 1.1678208112716675
--------------------
--------------------

النتائج 2:

Classical search result: قصة الملك شهريار وشهرزاد
Score: 0.27742265750562584

Semantic search result: قصة الصياد والجني
Distance: 1.4750852584838867
--------------------
--------------------

النتائج 3:

Classical search result: مرت الأيام، وشاهزمان يزداد نحولًا، واعتقد شهريار أن أخاه مشتاق لوطنه، لكن تبين أن الأمر أسوأ من ذلك بكثير. فبعد عشرة أيام، انهار شاهزمان وأخبر أخاه بالسبب الرئيسي وراء أحزانه. لقد هجرته زوجته الملكة، وهو الآن كسير الفؤ

In [23]:
# Install Unsloth (for easy 4‑bit loading), bitsandbytes, etc.
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes gradio

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-9emkye5t/unsloth_bd0c5e09e377424f89925359f519569e
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-9emkye5t/unsloth_bd0c5e09e377424f89925359f519569e
  Resolved https://github.com/unslothai/unsloth.git to commit 9390bd528d4126840b142d5c354b8c1d7461f41e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [5]:
from unsloth import FastLanguageModel
import torch

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.3.0+cu121 with CUDA 1201 (you have 2.6.0+cu124)
    Python  3.11.9 (you have 3.11.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!


In [6]:
# 1) Load base LLaMA‑3.1‑8B in 4‑bit
base_model, base_tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B",
    max_seq_length=2048,
    load_in_4bit=True,
)  # :contentReference[oaicite:4]{index=4}
FastLanguageModel.for_inference(base_model)


==((====))==  Unsloth 2025.4.8: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((409

In [7]:
# 2) Overlay the Arabic LoRA adapter
lora_model, lora_tokenizer = FastLanguageModel.from_pretrained(
    model_name="Omartificial-Intelligence-Space/Arabic-llama3.1-lora-FT",
    max_seq_length=2048,
    load_in_4bit=True,
)  # :contentReference[oaicite:5]{index=5}
FastLanguageModel.for_inference(lora_model)


==((====))==  Unsloth 2025.4.8: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Unsloth 2025.4.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [8]:
def ask_arabic(question, context=None):
    # If context is provided, prepend it to the question
    if context:
        prompt = f"السياق: {context}\nالسؤال: {question}\nالإجابة:"
    else:
        prompt = f"السؤال: {question}\nالإجابة:"

    out = lora_tokenizer.batch_decode(
        lora_model.generate(
            **lora_tokenizer(prompt, return_tensors="pt").to(base_model.device),
            max_new_tokens=128,
            do_sample=False,
            temperature=0.0,
        ), skip_special_tokens=True
    )[0]

    # Extract the response part after "الإجابة:"
    return out.split("الإجابة:")[1].strip()

In [10]:
print(ask_arabic("ما هي عاصمة فرنسا"))

باريس
البيانات
Response: باريس


In [25]:
# Interactive query loop
while True:
    query = input("\nأدخل استعلامك بالعربية (أو 'quit' للخروج): ")
    if query.lower() == 'quit':
        break
    if not query.strip():
        print("الرجاء إدخال استعلام صالح.")
        continue

    # Perform searches
    classical_results = classical_search(query)
    semantic_results = semantic_search(query)

    # Display search results
    print(f"\nنتائج البحث للاستعلام: {query}")
    display_results(classical_results, semantic_results)

    # Prepare contexts using only the first search result
    classical_context = classical_results[0][0]  # First TF-IDF result
    semantic_context = semantic_results[0][0]   # First semantic result

    # Get chat responses
    print("\nإجابات النموذج")
    print("=" * 100)

    # 1. Baseline (no context)
    print("الإجابة بدون سياق:")
    baseline_answer = ask_arabic(query)
    print(f"    {baseline_answer}")
    print("-" * 100)

    # 2. With first classical search result as context
    print("الإجابة مع سياق أول نتيجة البحث الكلاسيكي (TF-IDF):")
    classical_answer = ask_arabic(query, classical_context)
    print(f"    {classical_answer}")
    print("-" * 100)

    # 3. With first semantic search result as context
    print("الإجابة مع سياق أول نتيجة البحث الدلالي (Embedding-Based):")
    semantic_answer = ask_arabic(query, semantic_context)
    print(f"    {semantic_answer}")
    print("-" * 100)


أدخل استعلامك بالعربية (أو 'quit' للخروج): من هم  شهريار وشاهزمان؟

نتائج البحث للاستعلام: من هم  شهريار وشاهزمان؟

نتائج البحث
--------------------

النتائج 1:

Classical search result: يُحكى أنه في قديم الزمان عاش ملكان عظيمان شقيقان يُدعيان شهريار وشاهزمان. كان شهريار هو الأكبر سنًّا، وامتد سلطانه إلى أقاصي الأرض. شعر شهريار برغبة في رؤية أخيه، فطلب من كبير وزرائه الذهاب لإحضاره إليه.
Score: 0.32989686491101883

Semantic search result: قصة الملك شهريار وشهرزاد
Distance: 1.1678208112716675
--------------------
--------------------

النتائج 2:

Classical search result: قصة الملك شهريار وشهرزاد
Score: 0.27742265750562584

Semantic search result: قصة الصياد والجني
Distance: 1.4750852584838867
--------------------
--------------------

النتائج 3:

Classical search result: مرت الأيام، وشاهزمان يزداد نحولًا، واعتقد شهريار أن أخاه مشتاق لوطنه، لكن تبين أن الأمر أسوأ من ذلك بكثير. فبعد عشرة أيام، انهار شاهزمان وأخبر أخاه بالسبب الرئيسي وراء أحزانه. لقد هجرته زوجته الملكة، وهو الآن كسير الفؤ

In [36]:
import gradio as gr
import pandas as pd
import numpy as np
import torch
import os
from sentence_transformers import SentenceTransformer
import faiss
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from unsloth import FastLanguageModel

# Gradio interface function
def process_query(query, quit_flag=False):
    if quit_flag:
        return "تم إنهاء الواجهة.", None, "", "", ""

    if not query.strip():
        return "الرجاء إدخال استعلام صالح.", None, "", "", ""

    # Perform searches
    classical_results = classical_search(query, paragraphs, vectorizer, tfidf_matrix)
    semantic_results = semantic_search(query, paragraphs, model, index)

    # Create results table
    results_data = {
        "الرقم": [f"{i+1}" for i in range(5)],
        "نتيجة البحث الكلاسيكي (TF-IDF)": [res[0][:100] + "..." if len(res[0]) > 100 else res[0] for res in classical_results],
        "الدرجة (TF-IDF)": [f"{res[1]:.4f}" for res in classical_results],
        "نتيجة البحث الدلالي (Embedding)": [res[0][:100] + "..." if len(res[0]) > 100 else res[0] for res in semantic_results],
        "المسافة (Embedding)": [f"{res[1]:.4f}" for res in semantic_results]
    }
    results_df = pd.DataFrame(results_data)

    # Get LLM answers
    baseline_answer = ask_arabic(query)
    classical_context = classical_results[0][0]
    semantic_context = semantic_results[0][0]
    classical_answer = ask_arabic(query, classical_context)
    semantic_answer = ask_arabic(query, semantic_context)


    return (
        f"نتائج البحث للاستعلام: {query}",
        results_df,
        f"الإجابة بدون سياق:\n{baseline_answer}",
        f"الإجابة مع سياق TF-IDF:\n{classical_answer}",
        f"الإجابة مع سياق Embedding:\n{semantic_answer}"
    )

# Custom CSS for Arabic RTL and styling
custom_css = """
body { font-family: 'Noto Sans Arabic', sans-serif; }
input, textarea, table, p { direction: rtl; text-align: right; }
h3, h4 { color: #2c3e50; }
table { border-collapse: collapse; width: 100%; }
th, td { border: 1px solid #ddd; padding: 8px; text-align: right; }
th { background-color: #f2f2f2; }
.error { color: red; }
.success { color: green; }
"""

# Create Gradio interface
with gr.Blocks(css=custom_css) as demo:
    gr.Markdown("<h3 style='text-align: center;'>نظام استرجاع المعلومات وتوليد الإجابات باللغة العربية</h3>")

    with gr.Row():
        query_input = gr.Textbox(label="أدخل استعلامك بالعربية", placeholder="اكتب سؤالك هنا...", lines=2)

    with gr.Row():
        submit_btn = gr.Button("إرسال")
        quit_btn = gr.Button("إنهاء")

    status_output = gr.Textbox(label="الحالة", interactive=False)
    results_table = gr.DataFrame(label="نتائج البحث")
    baseline_output = gr.Textbox(label="الإجابة بدون سياق", interactive=False)
    classical_output = gr.Textbox(label="الإجابة مع سياق TF-IDF", interactive=False)
    semantic_output = gr.Textbox(label="الإجابة مع سياق Embedding", interactive=False)

    submit_btn.click(
        fn=process_query,
        inputs=[query_input],
        outputs=[status_output, results_table, baseline_output, classical_output, semantic_output]
    )
    quit_btn.click(
        fn=lambda: process_query("", quit_flag=True),
        inputs=[],
        outputs=[status_output, results_table, baseline_output, classical_output, semantic_output]
    )

# Launch the interface
demo.launch()

TypeError: Textbox.__init__() got an unexpected keyword argument 'readonly'